# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

demographic-person(__id__, gender, age)

medications-drug(__code__, name)

medications-use(__person_id__, __drug_code__, days_use)

    person_id -> CHE demographic-person
    drug_id -> CHE medications-drug

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

In [3]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [4]:
CREATE TABLE Drug (
  code VARCHAR(6) NOT NULL,
  name VARCHAR(60) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

In [5]:
CREATE TABLE Use (
  person_id VARCHAR(6) NOT NULL,
  drug_code VARCHAR(60) NOT NULL,
  days_use INTEGER,
  FOREIGN KEY(person_id)
    REFERENCES Pessoa(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(drug_code)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [6]:
SELECT * FROM Pessoa WHERE age > 60

b) Liste as idades das pessoas cadastradas (sem repetições)

In [7]:
SELECT DISTINCT(age) FROM Pessoa

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [8]:
SELECT p.id, p.age, d.code, u.days_use
FROM Pessoa p, Drug d, Use u
WHERE u.person_id = p.id AND u.drug_code = d.code

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [9]:
SELECT p.id, p.age, d.name, u.days_use
FROM Pessoa p, Drug d, Use u
WHERE u.person_id = p.id AND u.drug_code = d.code

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [10]:
SELECT DISTINCT(d.name)
FROM Use u, Drug d
WHERE u.drug_code = d.code

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [11]:
SELECT DISTINCT(d.name)
FROM Use u, Drug d
WHERE u.drug_code = d.code AND u.days_use > 2000